# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 9:13AM,243238,12,HH-36200,Hush Hush,Released
1,Aug 2 2022 9:13AM,243238,12,HH-36202,Hush Hush,Released
2,Aug 2 2022 9:13AM,243238,12,HH-36203,Hush Hush,Released
3,Aug 2 2022 9:13AM,243238,12,HH-36204,Hush Hush,Released
4,Aug 2 2022 9:13AM,243238,12,HH-36205,Hush Hush,Released
5,Aug 2 2022 9:13AM,243238,12,HH-36206,Hush Hush,Released
6,Aug 2 2022 9:13AM,243238,12,HH-36207,Hush Hush,Released
7,Aug 2 2022 9:13AM,243238,12,HH-36208,Hush Hush,Released
8,Aug 2 2022 9:13AM,243238,12,HH-36209,Hush Hush,Released
9,Aug 2 2022 9:13AM,243238,12,HH-36210,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,243234,Released,4
31,243235,Released,1
32,243236,Released,8
33,243237,Released,64
34,243238,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243234,NaN,4.0
243235,NaN,1.0
243236,NaN,8.0
243237,NaN,64.0
243238,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243129,0.0,1.0
243152,1.0,1.0
243156,0.0,3.0
243170,45.0,0.0
243185,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243129,0,1
243152,1,1
243156,0,3
243170,45,0
243185,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243129,0,1
1,243152,1,1
2,243156,0,3
3,243170,45,0
4,243185,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243129,,1
1,243152,1,1
2,243156,,3
3,243170,45,
4,243185,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 9:13AM,243238,12,Hush Hush
15,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation
79,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc."
81,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation
89,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc."
92,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated"
95,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc."
96,Aug 2 2022 8:30AM,243202,10,Emerginnova
98,Aug 2 2022 8:30AM,243215,10,Eywa Pharma Inc.
102,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 2 2022 9:13AM,243238,12,Hush Hush,,15
1,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,,64
2,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",1,1
3,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,,8
4,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",,3
5,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",,3
6,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",,1
7,Aug 2 2022 8:30AM,243202,10,Emerginnova,,2
8,Aug 2 2022 8:30AM,243215,10,Eywa Pharma Inc.,,4
9,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,4,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 2 2022 9:13AM,243238,12,Hush Hush,15,
1,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64,
2,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",1,1
3,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,
4,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3,
5,Aug 2 2022 8:34AM,243201,10,"Senseonics, Incorporated",3,
6,Aug 2 2022 8:30AM,243217,10,"Citieffe, Inc.",1,
7,Aug 2 2022 8:30AM,243202,10,Emerginnova,2,
8,Aug 2 2022 8:30AM,243215,10,Eywa Pharma Inc.,4,
9,Aug 2 2022 8:30AM,243205,10,ISDIN Corporation,2,4


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 2 2022 9:13AM,243238,12,Hush Hush,15,
1,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64,
2,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",1,1
3,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8,
4,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 2 2022 9:13AM,243238,12,Hush Hush,15.0,NaN
1,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64.0,NaN
2,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",1.0,1.0
3,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8.0,NaN
4,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 2 2022 9:13AM,243238,12,Hush Hush,15.0,0.0
1,Aug 2 2022 9:06AM,243237,10,ISDIN Corporation,64.0,0.0
2,Aug 2 2022 9:02AM,243227,20,"ACI Healthcare USA, Inc.",1.0,1.0
3,Aug 2 2022 8:43AM,243236,10,ISDIN Corporation,8.0,0.0
4,Aug 2 2022 8:37AM,243156,20,"ACI Healthcare USA, Inc.",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4134676,108.0,4.0
12,243238,15.0,0.0
16,243233,4.0,0.0
18,243232,1.0,0.0
19,1459128,24.0,3.0
20,729553,4.0,46.0
21,243218,1.0,0.0
22,243214,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4134676,108.0,4.0
1,12,243238,15.0,0.0
2,16,243233,4.0,0.0
3,18,243232,1.0,0.0
4,19,1459128,24.0,3.0
5,20,729553,4.0,46.0
6,21,243218,1.0,0.0
7,22,243214,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,108.0,4.0
1,12,15.0,0.0
2,16,4.0,0.0
3,18,1.0,0.0
4,19,24.0,3.0
5,20,4.0,46.0
6,21,1.0,0.0
7,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,108.0
1,12,Released,15.0
2,16,Released,4.0
3,18,Released,1.0
4,19,Released,24.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Executing,4.0,0.0,0.0,0.0,3.0,46.0,0.0,1.0
Released,108.0,15.0,4.0,1.0,24.0,4.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Executing,4.0,0.0,0.0,0.0,3.0,46.0,0.0,1.0
1,Released,108.0,15.0,4.0,1.0,24.0,4.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Executing,4.0,0.0,0.0,0.0,3.0,46.0,0.0,1.0
1,Released,108.0,15.0,4.0,1.0,24.0,4.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()